In [1]:
from collections import OrderedDict
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import rand, matmul, diag, einsum
from time import time

In [45]:
import sys

sys.path.append('../')

In [46]:
from model.resnet import *

In [2]:
batch_size = 16
num_channel = 16
arg_in_dim = 2
num_cell_types = 1
in_hidden_dim = 64


mlp_inputs = rand(batch_size,128)
conv_inputs = rand(batch_size, num_channel, 64, 64)

In [3]:
inner_net = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(arg_in_dim, in_hidden_dim)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(in_hidden_dim, in_hidden_dim)),
            ('relu2', nn.ReLU()),
            ('fc3', nn.Linear(in_hidden_dim, 1))]))

In [4]:
mlp_outputs = inner_net(mlp_inputs.reshape(batch_size, -1, arg_in_dim)).squeeze()

In [5]:
conv_inputs.reshape(batch_size, num_channel//arg_in_dim, -1, arg_in_dim).shape

torch.Size([16, 8, 4096, 2])

In [6]:
conv_outputs = inner_net(conv_inputs.reshape(batch_size, num_channel//arg_in_dim, -1, arg_in_dim)).squeeze()
conv_outputs = conv_outputs.reshape(batch_size, -1, 64, 64)

In [7]:
print(mlp_inputs.shape)
print(mlp_outputs.shape)

print(conv_inputs.shape)
print(conv_outputs.shape)

torch.Size([16, 128])
torch.Size([16, 64])
torch.Size([16, 16, 64, 64])
torch.Size([16, 8, 64, 64])


In [40]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset

In [41]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
test_dataset = datasets.CIFAR10(root='../xor_neuron_data/data',
                                train=False,
                                transform=transform,
                                download=False)

In [42]:
test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=batch_size,
                                 shuffle=False)

In [43]:
imgs, target = next(iter(test_loader))